In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'
# folder = 'data/gum/자동'
# folder = 'data/yeongsan1/자동'

# ## 한강
# ## 자동
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
#               ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
#               ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
#               ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
#               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
#               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = gum[0]
iloc_val

':,2:'

In [8]:
def make_columns(df):
    column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [9]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [10]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [11]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [12]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [13]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

가평_2016.xlsx
(8784, 12)
(8784, 12)
가평_2017.xlsx
(8744, 12)
(8760, 12)
가평_2018.xlsx
(8730, 12)
(8760, 12)
가평_2019.xlsx
(8760, 12)
(8760, 12)
의암호_2016.xlsx
(8784, 11)
(8784, 11)
의암호_2017.xlsx
(8760, 11)
(8760, 11)
의암호_2018.xlsx
(8746, 11)
(8760, 11)
의암호_2019.xlsx
(8760, 11)
(8760, 11)
서상_2016.xlsx
(8784, 11)
(8784, 11)
서상_2017.xlsx
(8760, 11)
(8760, 11)
서상_2018.xlsx
(8751, 11)
(8760, 11)
서상_2019.xlsx
(8760, 11)
(8760, 11)


In [14]:
train_mean, train_std

(수온          1.371826e+01
 수소이온농도      7.363691e+00
 전기전도도       1.062274e+02
 용존산소        1.048823e+01
 총유기탄소       1.599534e+00
 총질소         1.617681e+00
 총인          8.346780e-03
 클로로필-a      8.673920e+00
 Day sin     6.037103e-15
 Day cos     2.747682e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           7.365852
 수소이온농도       0.482657
 전기전도도       13.561701
 용존산소         2.071821
 총유기탄소        0.357371
 총질소          0.398078
 총인           0.007764
 클로로필-a       9.531307
 Day sin      0.707110
 Day cos      0.707110
 Year sin     0.707103
 Year cos     0.707117
 dtype: float64)

## 

In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0    19     3     0]
 [    3     8     1    19]
 [    4    12     2    27]
 ...
 [ 5314     9     9 17840]
 [ 5323     3     3 17849]
 [ 5326     6     1 17852]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,1.4,2.398,0.006,5.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,1.4,2.490,0.003,5.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,1.4,2.545,0.003,5.4,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,1.4,2.482,0.002,5.4,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,1.4,2.453,0.003,5.4,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,4.0,7.4,111.0,12.2,1.6,1.584,0.009,13.7,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.4,112.0,12.3,1.6,1.499,0.009,13.1,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.1,7.4,112.0,12.4,1.7,1.589,0.009,12.7,-7.071068e-01,0.707107,-0.004301,0.999991
35062,2.8,7.4,113.0,12.5,1.6,1.488,0.010,12.6,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  10270


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 116.5415 - disc_loss: 0.7208 - rmse: 1.0652 - val_loss: 0.9600
Epoch 2/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 95.1332 - disc_loss: 0.5250 - rmse: 0.9332 - val_loss: 0.8463
Epoch 3/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 87.3963 - disc_loss: 0.4836 - rmse: 0.9637 - val_loss: 0.8465
Epoch 4/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 71.4775 - disc_loss: 0.3834 - rmse: 0.8634 - val_loss: 0.7866
Epoch 5/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 57.8370 - disc_loss: 0.3895 - rmse: 0.7715 - val_loss: 0.7629
Epoch 6/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 52.3129 - disc_loss: 0.3897 - rmse: 0.7767 - val_loss: 0.7238
Epoch 7/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 43.8980 - disc_loss: 0.3731 - rmse: 0.6876 - val_loss: 0.6590
Ep

1/1 [==============================] - 0s 120ms/step - gen_loss: 6.5810 - disc_loss: 0.2922 - rmse: 0.3451 - val_loss: 0.3560
Epoch 60/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 8.7852 - disc_loss: 0.2923 - rmse: 0.3535 - val_loss: 0.3335
Epoch 61/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 6.3931 - disc_loss: 0.2981 - rmse: 0.3399 - val_loss: 0.3416
Epoch 62/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 5.9705 - disc_loss: 0.2916 - rmse: 0.3118 - val_loss: 0.3198
Epoch 63/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 5.7849 - disc_loss: 0.2992 - rmse: 0.3196 - val_loss: 0.2985
Epoch 64/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 6.5288 - disc_loss: 0.2908 - rmse: 0.3482 - val_loss: 0.3070
Epoch 65/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 6.1605 - disc_loss: 0.2937 - rmse: 0.3285 - val_loss: 0.3131
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 120ms/step - gen_loss: 4.7070 - disc_loss: 0.2421 - rmse: 0.2719 - val_loss: 0.2881
Epoch 118/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 5.2161 - disc_loss: 0.2331 - rmse: 0.3158 - val_loss: 0.2927
Epoch 119/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 4.9784 - disc_loss: 0.2357 - rmse: 0.2862 - val_loss: 0.2813
Epoch 120/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 4.8256 - disc_loss: 0.2363 - rmse: 0.2922 - val_loss: 0.2815
Epoch 121/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 4.6307 - disc_loss: 0.2351 - rmse: 0.3063 - val_loss: 0.3005
Epoch 122/2000
1/1 [==============================] - 0s 111ms/step - gen_loss: 4.7102 - disc_loss: 0.2353 - rmse: 0.2752 - val_loss: 0.2903
Epoch 123/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 4.4041 - disc_loss: 0.2289 - rmse: 0.2846 - val_loss: 0.2931
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 141ms/step - gen_loss: 4.8445 - disc_loss: 0.2155 - rmse: 0.2918 - val_loss: 0.2707
Epoch 176/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 4.3448 - disc_loss: 0.2091 - rmse: 0.2562 - val_loss: 0.2567
Epoch 177/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 6.5728 - disc_loss: 0.2140 - rmse: 0.2560 - val_loss: 0.2680
Epoch 178/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 4.7874 - disc_loss: 0.2162 - rmse: 0.2744 - val_loss: 0.2872
Epoch 179/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 4.6066 - disc_loss: 0.2025 - rmse: 0.2677 - val_loss: 0.2518
Epoch 180/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 4.3858 - disc_loss: 0.2072 - rmse: 0.2761 - val_loss: 0.2548
Epoch 181/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 4.7617 - disc_loss: 0.2073 - rmse: 0.2747 - val_loss: 0.2676
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 117ms/step - gen_loss: 3.9117 - disc_loss: 0.1877 - rmse: 0.2472 - val_loss: 0.2787
Epoch 234/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 3.5315 - disc_loss: 0.1898 - rmse: 0.2380 - val_loss: 0.2583
Epoch 235/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 3.9986 - disc_loss: 0.1898 - rmse: 0.2542 - val_loss: 0.2717
Epoch 236/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 3.7688 - disc_loss: 0.1922 - rmse: 0.2489 - val_loss: 0.2388
Epoch 237/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 3.9832 - disc_loss: 0.1878 - rmse: 0.2588 - val_loss: 0.2512
Epoch 238/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 4.1390 - disc_loss: 0.1877 - rmse: 0.2540 - val_loss: 0.2693
Epoch 239/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 4.4553 - disc_loss: 0.1957 - rmse: 0.2626 - val_loss: 0.2410
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 124ms/step - gen_loss: 2.8339 - disc_loss: 0.1375 - rmse: 0.2302 - val_loss: 0.2158
Epoch 930/2000
1/1 [==============================] - 0s 160ms/step - gen_loss: 2.7783 - disc_loss: 0.1349 - rmse: 0.2415 - val_loss: 0.2617
Epoch 931/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 3.5367 - disc_loss: 0.1324 - rmse: 0.2637 - val_loss: 0.2572
Epoch 932/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 2.6790 - disc_loss: 0.1358 - rmse: 0.2369 - val_loss: 0.2182
Epoch 933/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 2.7979 - disc_loss: 0.1316 - rmse: 0.2230 - val_loss: 0.2364
Epoch 934/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 3.3925 - disc_loss: 0.1301 - rmse: 0.3000 - val_loss: 0.2512
Epoch 935/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 2.6805 - disc_loss: 0.1337 - rmse: 0.2305 - val_loss: 0.2220
Epoch 936/2000
1/1 [========

1/1 [==============================] - 0s 120ms/step - gen_loss: 2.6863 - disc_loss: 0.1340 - rmse: 0.2217 - val_loss: 0.2303
Epoch 988/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 2.7129 - disc_loss: 0.1326 - rmse: 0.2270 - val_loss: 0.2377
Epoch 989/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.9717 - disc_loss: 0.1298 - rmse: 0.2457 - val_loss: 0.2185
Epoch 990/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 2.6080 - disc_loss: 0.1315 - rmse: 0.2172 - val_loss: 0.2501
Epoch 991/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 3.2294 - disc_loss: 0.1338 - rmse: 0.2412 - val_loss: 0.2281
Epoch 992/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 4.7086 - disc_loss: 0.1297 - rmse: 0.2360 - val_loss: 0.2267
Epoch 993/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.8292 - disc_loss: 0.1291 - rmse: 0.2229 - val_loss: 0.2386
Epoch 994/2000
1/1 [========

Epoch 1045/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 3.6362 - disc_loss: 0.1323 - rmse: 0.2441 - val_loss: 0.2343
Epoch 1046/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 2.8568 - disc_loss: 0.1329 - rmse: 0.2202 - val_loss: 0.2739
Epoch 1047/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 3.0880 - disc_loss: 0.1293 - rmse: 0.2437 - val_loss: 0.2576
Epoch 1048/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 3.4217 - disc_loss: 0.1308 - rmse: 0.2378 - val_loss: 0.2372
Epoch 1049/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 3.5222 - disc_loss: 0.1315 - rmse: 0.2606 - val_loss: 0.2798
Epoch 1050/2000
1/1 [==============================] - 0s 110ms/step - gen_loss: 4.6365 - disc_loss: 0.1335 - rmse: 0.2506 - val_loss: 0.2576
Epoch 1051/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 3.1584 - disc_loss: 0.1285 - rmse: 0.2451 - val_loss: 0.2576
Epoch 

1/1 [==============================] - 0s 106ms/step - gen_loss: 2.4832 - disc_loss: 0.1279 - rmse: 0.2093 - val_loss: 0.2500
Epoch 1103/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 2.4739 - disc_loss: 0.1272 - rmse: 0.2035 - val_loss: 0.2658
Epoch 1104/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 2.5644 - disc_loss: 0.1305 - rmse: 0.2279 - val_loss: 0.2216
Epoch 1105/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 2.3877 - disc_loss: 0.1285 - rmse: 0.2069 - val_loss: 0.2374
Epoch 1106/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 3.1944 - disc_loss: 0.1264 - rmse: 0.2022 - val_loss: 0.2484
Epoch 1107/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 2.4065 - disc_loss: 0.1290 - rmse: 0.2133 - val_loss: 0.2433
Epoch 1108/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.4049 - disc_loss: 0.1245 - rmse: 0.2071 - val_loss: 0.2264
Epoch 1109/2000
1/1 [=

1/1 [==============================] - 0s 129ms/step - gen_loss: 3.3349 - disc_loss: 0.1298 - rmse: 0.2379 - val_loss: 0.2544
Epoch 1160/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 2.4376 - disc_loss: 0.1312 - rmse: 0.2030 - val_loss: 0.2529
Epoch 1161/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 2.4658 - disc_loss: 0.1252 - rmse: 0.2143 - val_loss: 0.2267
Epoch 1162/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.6104 - disc_loss: 0.1278 - rmse: 0.2161 - val_loss: 0.2426
Epoch 1163/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 2.4759 - disc_loss: 0.1255 - rmse: 0.2000 - val_loss: 0.2344
Epoch 1164/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 3.2873 - disc_loss: 0.1263 - rmse: 0.2330 - val_loss: 0.2647
Epoch 1165/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 2.5108 - disc_loss: 0.1271 - rmse: 0.2320 - val_loss: 0.2299
Epoch 1166/2000
1/1 [=

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 30ms/step - loss: 0.2596


0.25960788130760193

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 12)
120
x.shape = (105120, 12)
x_reshape.shape = (876, 120, 12)
143409
y_true.shape= (105120, 12)
143409


(1, 120, 12)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 12) (120, 12)
(105192, 12)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 12)
120
n= 105120
(35040, 8)
(35040, 8)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0     -1.251486  0.282415  1.457973  0.150479 -0.558340  1.960215 -0.302267   
1     -1.278638  0.282415  1.457973  0.150479 -0.558340  2.191326 -0.688669   
2     -1.265062  0.282415  1.457973  0.150479 -0.558340  2.329489 -0.688669   
3     -1.265062  0.282415  1.457973  0.150479 -0.558340  2.171229 -0.817470   
4     -1.265062  0.282415  1.457973  0.150479 -0.558340  2.098379 -0.688669   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -1.373671 -0.546331  0.646865  1.019280  0.560945  0.252009 -0.817470   
35060 -1.387248 -0.546331  0.646865  1.019280  0.281124  0.272106 -0.946270   
35061 -1.400824 -0.546331  0.646865  1.019280  0.281124  0.302251 -0.817470   
35062 -1.400824 -0.546331  0.646865  1.019280  0.281124  0.264570 -0.817470   
35063 -1.414400 -0.546331  0.646865  1.019280  0.281124  0.219352 -0.817470   

         클로로필-a      

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.8)
val_no = int(total_no * 0.9)

train_slice = slice(0, train_no)
val_slice = slice(train_no, val_no)
test_slice = slice(val_no, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [53]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-1.251486,0.282415,1.457973,0.150479,-0.558340,1.960215,-0.302267,-0.312016,-1.999586e-12,1.414207,...,0.187975,-0.238574,-1.265062,0.903974,-0.016769,1.067547,-0.838161,-0.407285,0.230893,-0.616276
1,-1.278638,0.282415,1.457973,0.150479,-0.558340,2.191326,-0.688669,-0.343491,3.660237e-01,1.366019,...,0.073602,-0.144148,-1.292214,0.903974,-0.016769,1.067547,-0.838161,-0.352897,0.107308,-0.626768
2,-1.265062,0.282415,1.457973,0.150479,-0.558340,2.329489,-0.688669,-0.343491,7.071034e-01,1.224739,...,0.295861,-0.196607,-1.332943,0.903974,0.056968,0.971013,-0.838161,-0.142621,0.119231,-0.626768
3,-1.265062,0.282415,1.457973,0.150479,-0.558340,2.171229,-0.817470,-0.343491,9.999952e-01,0.999995,...,0.065565,-0.343491,-1.373671,0.696788,0.056968,0.971013,-0.838161,-0.254271,0.090900,-0.616276
4,-1.265062,0.282415,1.457973,0.150479,-0.558340,2.098379,-0.688669,-0.343491,1.224739e+00,0.707103,...,-0.032882,-0.312600,-1.400824,0.696788,-0.016769,0.971013,-0.838161,-0.257631,0.029084,-0.637260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28027,-1.115724,-0.339145,1.163024,0.922746,-0.278518,0.528337,0.084135,0.286013,-1.366019e+00,0.366024,...,-0.173466,0.842076,-1.183605,0.903974,0.499391,1.357147,-0.558340,-0.911078,0.341736,-0.374966
28028,-1.129300,-0.339145,1.163024,0.922746,0.001303,0.515776,0.084135,0.265030,-1.224739e+00,0.707103,...,-0.173466,0.821092,-1.197181,0.903974,0.499391,1.357147,-0.558340,-0.896006,0.212936,-0.395950
28029,-1.129300,-0.339145,1.163024,0.922746,-0.278518,0.533361,-0.044666,0.254538,-9.999952e-01,0.999995,...,-0.173466,0.863059,-1.251486,0.903974,0.573128,1.357147,-0.558340,-0.923638,0.084135,-0.406442
28030,-1.142877,-0.339145,1.163024,0.922746,-0.278518,0.493168,-0.044666,0.244046,-7.071034e-01,1.224739,...,-0.302267,0.946993,-1.292214,0.696788,0.573128,1.357147,-0.558340,-0.923638,0.084135,-0.395950


In [54]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [55]:
MAX_EPOCHS = 150
target_col = '용존산소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
# out_features = [target_col_idx]
out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

5

In [56]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [57]:
OUT_STEPS = 24*5

In [58]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [59]:
# train_df

In [60]:
# it = iter(wdg)

In [61]:
# x,y = next(it)
# x.shape, y.shape

In [62]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [63]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [64]:
def hour_to_day_mean(array):
    
#     print("1")
#     print(array)
    
    time = 24
    array = np.reshape(array, (array.shape[0], array.shape[1]//time, time, array.shape[2]))

    array = np.mean(array, axis=2)
    
#     print(array)

    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [66]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
#     plot_out_col_index = self.column_indices[plot_out_col]
    
#     if self.label_columns:
#         label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
#     else:
#         label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
#     print(plot_col)
    
#     print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    sum_pred = 0
    sum_label =0
    
    o = 0
    p = 0
    
#     print(train_std[plot_col])
    
#     print(inputs.shape)
#     print(inputs[0].shape)

#     inputs = inputs[0:1]
    
# #     print(inputs)
    
#     labels = labels[0:1]
    
    predictions = model(inputs)
    
    pred = predictions.numpy()
    label = labels.numpy()
    
    for i in range(5) :
#         print(print("column : " + df_all.columns[plot_col[i]]))
        pred[:,:,i] = pred[:,:,i] * train_std[plot_col[i]] + train_mean[plot_col[i]]
        label[:,:,i] = label[:,:,i]* train_std[plot_col[i]] + train_mean[plot_col[i]]

        
#     pred_temp = hour_to_day_mean(pred[127:128,:,:]) 
    #for n in range(len(inputs)):
    #     for i in range(5) :         
     #           pred_temp = hour_to_day_mean(pred[n:n+1,:,i:i+1])
     #           label_temp = hour_to_day_mean(label[n:n+1,:,i:i+1])
    
     #for n in range(len(inputs)):           
      #      pred_temp = hour_to_day_mean(pred[n:n+1,:,:])
      #      label_temp = hour_to_day_mean(label[n:n+1,:,:])
            
#     print(pred.shape)
    
    pred = hour_to_day_mean(pred)
    label = hour_to_day_mean(label)
    
    o = np.sum(label[:,4,:],axis=0)
    
    p = np.sum(pred[:,4,:],axis=0)
    
    print("p")
    print(p)
    print("o")
    print(o)
    
       
    err = np.abs(o-p) / o *100
    
#     print(err.shape)
    
#     print(err)
    
    print(err[0])
    print(err[1])
    print(err[2])
    print(err[3])
    print(err[4])
    
    
#     print(np.sum(err,axis=0))
    
    
        
        
#     print(pred_temp)
#     np.sum(pred_temp)
#     print(pred_temp.shape)
    
#     label_temp = hour_to_day_mean(label[0][:][0])
#     print(pred[0,:,0])
#     print("adsfasdfadsfadsf")
#     print(pred_temp)
#     print(pred[0,0,0])
#     print(pred[0,0,1])
#     print(pred[0,0,2])
#     print(pred[0,0,3])
#     print(pred[0,0,4])
#       pred_temp = hour_to_day_mean(pred[0][:][0]) 
#     label_temp = hour_to_day_mean(label[0][:][0])
        
#     print(pred_temp[4])
#     print(pred_temp[4])
    
#     for n in range(len(inputs)):
#         pred_temp = hour_to_day_mean(pred[n][:][0]) 
#         label_temp = hour_to_day_mean(label[n][:][0])
        
#         print(pred_temp[4])
#         print(pred_temp[4])
    
    
#     predictions[:][:][pred_col_no] = predictions[:][:][pred_col_no] * train_std[plot_col] + train_mean[plot_col]
#     labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions[0])
# #     print(plot_col)
# #     print(plot_out_col_index)
# #     pred = predictions.numpy()
# #     print(pred.shape)
# #     print(pred)
# #     print(pred[0][0][pred_col_no])

#     label = labels.numpy()
#     print(label.shape)
#     print(label)
#     print(label[0][0][pred_col_no])
    
    
    
#     print('++++++++++++++++++++++++')
#     print(np.sum(predictions.numpy()))
    
#     print(predictions.numpy().sum())
    
#     print('++++++++++++++++++++++++')
#     print(np.sum(labels.numpy()))
    
#     o = np.sum(labels.numpy())
#     p = np.sum(predictions.numpy())
#     print('++++++++++++++++++++++++')
#     print(np.abs(o-p) / o *100)

    
    
    
#     if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
#         predictions = predictions.numpy()
#         labels = labels.numpy()
        
#         predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
#         labels = (labels - labels.min()) / (labels.max() - labels.min())
    
#     for n in range(len(inputs)):
#         pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
#         label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
        
# #         print(pred_temp[4].numpy())

        
#         sum_pred = sum_pred + pred_temp[4].numpy()
#         sum_label = sum_label + label_temp[4].numpy()
       
        
#     mae = mean_absolute_error(label_arr, pred_arr) 
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
   

# #     print(sum_pred)
#     pbias = ( np.abs(sum_label - sum_pred) / sum_label ) * 100 
    
# #     pbias = ( (sum_label - sum_pred) / sum_label ) * 100 
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
#     print('pbias')
#     print(pbias)
    
    

WindowGenerator.compa = compa

In [67]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [68]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [69]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [70]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [71]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [72]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [73]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [74]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [75]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 795ms/step - loss: 0.6874 - mean_absolute_error: 0.6012 - val_loss: 0.5329 - val_mean_absolute_error: 0.6282
Epoch 2/150
1/1 [==============================] - 0s 297ms/step - loss: 0.8476 - mean_absolute_error: 0.6164 - val_loss: 0.4985 - val_mean_absolute_error: 0.6051
Epoch 3/150
1/1 [==============================] - 0s 287ms/step - loss: 0.8020 - mean_absolute_error: 0.6131 - val_loss: 0.4724 - val_mean_absolute_error: 0.5928
Epoch 4/150
1/1 [==============================] - 0s 275ms/step - loss: 0.6005 - mean_absolute_error: 0.5687 - val_loss: 0.3931 - val_mean_absolute_error: 0.5275
Epoch 5/150
1/1 [==============================] - 0s 289ms/step - loss: 0.7157 - mean_absolute_error: 0.5982 - val_loss: 0.4573 - val_mean_absolute_error: 0.5785
Epoch 6/150
1/1 [==============================] - 0s 284ms/step - loss: 0.6795 - mean_absolute_error: 0.5565 - val_loss: 0.3668 - val_mean_absolute_error: 0.5186
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 271ms/step - loss: 0.1476 - mean_absolute_error: 0.2709 - val_loss: 0.2189 - val_mean_absolute_error: 0.3697
Epoch 52/150
1/1 [==============================] - 0s 275ms/step - loss: 0.2068 - mean_absolute_error: 0.2976 - val_loss: 0.2381 - val_mean_absolute_error: 0.3815
Epoch 53/150
1/1 [==============================] - 0s 255ms/step - loss: 0.1888 - mean_absolute_error: 0.2707 - val_loss: 0.2237 - val_mean_absolute_error: 0.3720
Epoch 54/150
1/1 [==============================] - 0s 273ms/step - loss: 0.1944 - mean_absolute_error: 0.2833 - val_loss: 0.2015 - val_mean_absolute_error: 0.3554
Epoch 55/150
1/1 [==============================] - 0s 294ms/step - loss: 0.2142 - mean_absolute_error: 0.2928 - val_loss: 0.2154 - val_mean_absolute_error: 0.3709
Epoch 56/150
1/1 [==============================] - 0s 265ms/step - loss: 0.1656 - mean_absolute_error: 0.2637 - val_loss: 0.2116 - val_mean_absolute_error: 0.3656
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 328ms/step - loss: 0.1402 - mean_absolute_error: 0.2514 - val_loss: 0.1711 - val_mean_absolute_error: 0.3222
Epoch 102/150
1/1 [==============================] - 0s 291ms/step - loss: 0.1867 - mean_absolute_error: 0.2361 - val_loss: 0.1847 - val_mean_absolute_error: 0.3335
Epoch 103/150
1/1 [==============================] - 0s 295ms/step - loss: 0.1122 - mean_absolute_error: 0.2280 - val_loss: 0.1811 - val_mean_absolute_error: 0.3273
Epoch 104/150
1/1 [==============================] - 0s 308ms/step - loss: 0.1203 - mean_absolute_error: 0.2345 - val_loss: 0.1558 - val_mean_absolute_error: 0.3122
Epoch 105/150
1/1 [==============================] - 0s 262ms/step - loss: 0.1970 - mean_absolute_error: 0.2348 - val_loss: 0.1827 - val_mean_absolute_error: 0.3301
Epoch 106/150
1/1 [==============================] - 0s 277ms/step - loss: 0.1192 - mean_absolute_error: 0.2248 - val_loss: 0.1886 - val_mean_absolute_error: 0.3343
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 93ms/step - loss: 0.1237 - mean_absolute_error: 0.2517
val performance = [0.17776088416576385, 0.3231743574142456]
test performance =  [0.12367148697376251, 0.25166603922843933]


In [76]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [78]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [79]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.7932 - mean_absolute_error: 0.6174 - val_loss: 0.5442 - val_mean_absolute_error: 0.6458
Epoch 2/150
1/1 [==============================] - 0s 293ms/step - loss: 0.6033 - mean_absolute_error: 0.5742 - val_loss: 0.4792 - val_mean_absolute_error: 0.6069
Epoch 3/150
1/1 [==============================] - 0s 287ms/step - loss: 0.7794 - mean_absolute_error: 0.6015 - val_loss: 0.5061 - val_mean_absolute_error: 0.6055
Epoch 4/150
1/1 [==============================] - 0s 336ms/step - loss: 0.6453 - mean_absolute_error: 0.5677 - val_loss: 0.5045 - val_mean_absolute_error: 0.6135
Epoch 5/150
1/1 [==============================] - 0s 282ms/step - loss: 0.6831 - mean_absolute_error: 0.5642 - val_loss: 0.5165 - val_mean_absolute_error: 0.6166
Epoch 6/150
1/1 [==============================] - 0s 319ms/step - loss: 0.7271 - mean_absolute_error: 0.6015 - val_loss: 0.4551 - val_mean_absolute_error: 0.5724
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 317ms/step - loss: 0.3385 - mean_absolute_error: 0.3247 - val_loss: 0.1823 - val_mean_absolute_error: 0.3258
Epoch 52/150
1/1 [==============================] - 0s 313ms/step - loss: 0.3080 - mean_absolute_error: 0.3300 - val_loss: 0.1944 - val_mean_absolute_error: 0.3302
Epoch 53/150
1/1 [==============================] - 0s 319ms/step - loss: 0.2102 - mean_absolute_error: 0.2929 - val_loss: 0.2114 - val_mean_absolute_error: 0.3346
Epoch 54/150
1/1 [==============================] - 0s 305ms/step - loss: 0.1877 - mean_absolute_error: 0.3017 - val_loss: 0.1794 - val_mean_absolute_error: 0.3175
Epoch 55/150
1/1 [==============================] - 0s 278ms/step - loss: 0.2046 - mean_absolute_error: 0.2853 - val_loss: 0.2466 - val_mean_absolute_error: 0.3543
Epoch 56/150
1/1 [==============================] - 0s 316ms/step - loss: 0.2125 - mean_absolute_error: 0.3044 - val_loss: 0.2083 - val_mean_absolute_error: 0.3341
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 316ms/step - loss: 0.1935 - mean_absolute_error: 0.2572 - val_loss: 0.1543 - val_mean_absolute_error: 0.3078
Epoch 102/150
1/1 [==============================] - 0s 313ms/step - loss: 0.1405 - mean_absolute_error: 0.2549 - val_loss: 0.1436 - val_mean_absolute_error: 0.2998
Epoch 103/150
1/1 [==============================] - 0s 324ms/step - loss: 0.1250 - mean_absolute_error: 0.2426 - val_loss: 0.1340 - val_mean_absolute_error: 0.2913
Epoch 104/150
1/1 [==============================] - 0s 330ms/step - loss: 0.1448 - mean_absolute_error: 0.2486 - val_loss: 0.1752 - val_mean_absolute_error: 0.3141
Epoch 105/150
1/1 [==============================] - 0s 323ms/step - loss: 0.1257 - mean_absolute_error: 0.2430 - val_loss: 0.1783 - val_mean_absolute_error: 0.3192
Epoch 106/150
1/1 [==============================] - 0s 305ms/step - loss: 0.1671 - mean_absolute_error: 0.2501 - val_loss: 0.1680 - val_mean_absolute_error: 0.3114
Epoch 107/150
1/1 [=====

100/100 [==============================] - 10s 101ms/step - loss: 0.1332 - mean_absolute_error: 0.2643
val performance = [0.13256363570690155, 0.283077597618103]
test performance =  [0.13316012918949127, 0.264283150434494]


**LSTM**

In [80]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [81]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 3s 3s/step - loss: 0.8739 - mean_absolute_error: 0.6389 - val_loss: 0.5279 - val_mean_absolute_error: 0.6236
Epoch 2/150
1/1 [==============================] - 0s 232ms/step - loss: 0.7597 - mean_absolute_error: 0.6085 - val_loss: 0.4688 - val_mean_absolute_error: 0.5935
Epoch 3/150
1/1 [==============================] - 0s 270ms/step - loss: 0.7226 - mean_absolute_error: 0.5956 - val_loss: 0.4931 - val_mean_absolute_error: 0.6094
Epoch 4/150
1/1 [==============================] - 0s 267ms/step - loss: 0.6576 - mean_absolute_error: 0.5997 - val_loss: 0.4583 - val_mean_absolute_error: 0.5860
Epoch 5/150
1/1 [==============================] - 0s 242ms/step - loss: 0.5984 - mean_absolute_error: 0.5646 - val_loss: 0.4632 - val_mean_absolute_error: 0.5815
Epoch 6/150
1/1 [==============================] - 0s 253ms/step - loss: 0.5949 - mean_absolute_error: 0.5345 - val_loss: 0.4744 - val_mean_absolute_error: 0.5948
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 258ms/step - loss: 0.2567 - mean_absolute_error: 0.2830 - val_loss: 0.2114 - val_mean_absolute_error: 0.3425
Epoch 52/150
1/1 [==============================] - 0s 262ms/step - loss: 0.2000 - mean_absolute_error: 0.2939 - val_loss: 0.1666 - val_mean_absolute_error: 0.3186
Epoch 53/150
1/1 [==============================] - 0s 273ms/step - loss: 0.1950 - mean_absolute_error: 0.2923 - val_loss: 0.2136 - val_mean_absolute_error: 0.3489
Epoch 54/150
1/1 [==============================] - 0s 262ms/step - loss: 0.2320 - mean_absolute_error: 0.2930 - val_loss: 0.1802 - val_mean_absolute_error: 0.3303
Epoch 55/150
1/1 [==============================] - 0s 233ms/step - loss: 0.2055 - mean_absolute_error: 0.2987 - val_loss: 0.2100 - val_mean_absolute_error: 0.3433
Epoch 56/150
1/1 [==============================] - 0s 251ms/step - loss: 0.2522 - mean_absolute_error: 0.3060 - val_loss: 0.1993 - val_mean_absolute_error: 0.3416
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 244ms/step - loss: 0.2287 - mean_absolute_error: 0.2516 - val_loss: 0.1844 - val_mean_absolute_error: 0.3447
Epoch 102/150
1/1 [==============================] - 0s 255ms/step - loss: 0.1719 - mean_absolute_error: 0.2416 - val_loss: 0.1666 - val_mean_absolute_error: 0.3294
Epoch 103/150
1/1 [==============================] - 0s 231ms/step - loss: 0.1197 - mean_absolute_error: 0.2188 - val_loss: 0.1851 - val_mean_absolute_error: 0.3480
Epoch 104/150
1/1 [==============================] - 0s 246ms/step - loss: 0.1396 - mean_absolute_error: 0.2390 - val_loss: 0.1649 - val_mean_absolute_error: 0.3334
Epoch 105/150
1/1 [==============================] - 0s 268ms/step - loss: 0.1855 - mean_absolute_error: 0.2498 - val_loss: 0.1696 - val_mean_absolute_error: 0.3294
Epoch 106/150
1/1 [==============================] - 0s 273ms/step - loss: 0.1485 - mean_absolute_error: 0.2333 - val_loss: 0.1857 - val_mean_absolute_error: 0.3483
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 133ms/step - loss: 0.1715 - mean_absolute_error: 0.3354
val performance = [0.17149677872657776, 0.3353668749332428]
test performance =  [0.19901438057422638, 0.33191993832588196]


**CNN model**

In [82]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [83]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 838ms/step - loss: 0.6906 - mean_absolute_error: 0.6030 - val_loss: 0.5285 - val_mean_absolute_error: 0.6342
Epoch 2/150
1/1 [==============================] - 0s 223ms/step - loss: 0.6704 - mean_absolute_error: 0.5869 - val_loss: 0.4766 - val_mean_absolute_error: 0.5970
Epoch 3/150
1/1 [==============================] - 0s 245ms/step - loss: 0.7108 - mean_absolute_error: 0.5698 - val_loss: 0.4235 - val_mean_absolute_error: 0.5351
Epoch 4/150
1/1 [==============================] - 0s 250ms/step - loss: 0.6099 - mean_absolute_error: 0.5191 - val_loss: 0.3978 - val_mean_absolute_error: 0.5102
Epoch 5/150
1/1 [==============================] - 0s 247ms/step - loss: 0.4650 - mean_absolute_error: 0.4625 - val_loss: 0.3959 - val_mean_absolute_error: 0.4916
Epoch 6/150
1/1 [==============================] - 0s 241ms/step - loss: 0.4474 - mean_absolute_error: 0.4583 - val_loss: 0.3763 - val_mean_absolute_error: 0.4984
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 233ms/step - loss: 0.1465 - mean_absolute_error: 0.2468 - val_loss: 0.1523 - val_mean_absolute_error: 0.2998
Epoch 52/150
1/1 [==============================] - 0s 215ms/step - loss: 0.1915 - mean_absolute_error: 0.2512 - val_loss: 0.1424 - val_mean_absolute_error: 0.2990
Epoch 53/150
1/1 [==============================] - 0s 235ms/step - loss: 0.2704 - mean_absolute_error: 0.2611 - val_loss: 0.1709 - val_mean_absolute_error: 0.3114
Epoch 54/150
1/1 [==============================] - 0s 246ms/step - loss: 0.1049 - mean_absolute_error: 0.2211 - val_loss: 0.1574 - val_mean_absolute_error: 0.3008
Epoch 55/150
1/1 [==============================] - 0s 236ms/step - loss: 0.1259 - mean_absolute_error: 0.2390 - val_loss: 0.1373 - val_mean_absolute_error: 0.2832
Epoch 56/150
1/1 [==============================] - 0s 228ms/step - loss: 0.1514 - mean_absolute_error: 0.2326 - val_loss: 0.1567 - val_mean_absolute_error: 0.2935
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 251ms/step - loss: 0.1811 - mean_absolute_error: 0.2326 - val_loss: 0.1845 - val_mean_absolute_error: 0.3300
Epoch 102/150
1/1 [==============================] - 0s 254ms/step - loss: 0.1197 - mean_absolute_error: 0.2281 - val_loss: 0.1680 - val_mean_absolute_error: 0.3140
Epoch 103/150
1/1 [==============================] - 0s 222ms/step - loss: 0.1729 - mean_absolute_error: 0.2304 - val_loss: 0.1891 - val_mean_absolute_error: 0.3256
Epoch 104/150
1/1 [==============================] - 0s 268ms/step - loss: 0.1663 - mean_absolute_error: 0.2272 - val_loss: 0.1723 - val_mean_absolute_error: 0.3200
Epoch 105/150
1/1 [==============================] - 0s 245ms/step - loss: 0.1570 - mean_absolute_error: 0.2224 - val_loss: 0.1689 - val_mean_absolute_error: 0.3213
Epoch 106/150
1/1 [==============================] - 0s 234ms/step - loss: 0.1419 - mean_absolute_error: 0.2247 - val_loss: 0.1863 - val_mean_absolute_error: 0.3260
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 111ms/step - loss: 0.2422 - mean_absolute_error: 0.3870
val performance = [0.24222330749034882, 0.3869689404964447]
test performance =  [0.13175292313098907, 0.27009499073028564]


In [84]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 125ms/step - loss: 0.2157 - mean_absolute_error: 0.3709
val performance = [0.215741828083992, 0.3708533048629761]
test performance =  [0.11341629177331924, 0.2569439113140106]


**performance**

In [85]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
multi_window.compa(elman_model, plot_col=out_features, windows=multi_window.example2)

p
[1.2485096e+03 2.2393414e+02 2.2220020e+02 6.4854652e-01 1.4035059e+03]
o
[1.2596379e+03 2.2033943e+02 1.9675500e+02 3.7891734e-01 1.2037831e+03]
0.88345194
1.6314427
12.932423
71.15778
16.591259
